In [1]:
# 02_model_experimentation.ipynb - Cell 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys
import joblib
warnings.filterwarnings('ignore')

# Add project to path
project_path = r"C:\Users\Icey_m_a\Documents\Icey\Icey\School\Python\Employee Attrition Prediction Model"
if project_path not in sys.path:
    sys.path.append(project_path)

# Import preprocessing
from src.data.preprocessing import load_raw_data, create_preprocessing_pipeline, prepare_data

# ML libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                           roc_auc_score, confusion_matrix, classification_report)
import mlflow
import mlflow.sklearn

print("✅ Setup complete")
print(f"✅ MLflow version: {mlflow.__version__}")

✅ Setup complete
✅ MLflow version: 3.9.0


In [2]:
# Cell 2 - Load and prepare data
print("📂 Loading data...")
df = load_raw_data()

print("\n🔧 Creating preprocessing pipeline...")
preprocessor, drop_cols, num_cols, cat_cols = create_preprocessing_pipeline()

print("\n🔄 Preparing train/test split...")
X_train, X_test, y_train, y_test, feature_names, fitted_preprocessor = prepare_data(
    df, preprocessor, drop_cols, test_size=0.2, random_state=42
)

# Save preprocessor for later use
os.makedirs('../models', exist_ok=True)
joblib.dump(fitted_preprocessor, '../models/preprocessor.pkl')
print("\n✅ Preprocessor saved to ../models/preprocessor.pkl")

📂 Loading data...
✅ Dataset loaded successfully!
   Shape: (1470, 35)
   Attrition distribution:
Attrition
No     0.838776
Yes    0.161224
Name: proportion, dtype: float64

🔧 Creating preprocessing pipeline...
✅ Preprocessing pipeline created
   Numerical features: 23
   Categorical features: 7
   Columns to drop: 4

🔄 Preparing train/test split...

🔄 Preparing data (train=80%, test=20%)...
   Features shape: (1470, 30)
   Target distribution: {0: 1233, 1: 237}
   Train set: 1176 samples
   Test set: 294 samples
   Applying preprocessing...
✅ Data preparation complete!
   Processed train shape: (1176, 44)
   Processed test shape: (294, 44)
   Total features: 44

✅ Preprocessor saved to ../models/preprocessor.pkl


In [ ]:
# Cell 3 - Train and compare multiple models
print("=" * 60)
print("🤖 MODEL TRAINING WITH MLFLOW TRACKING")
print("=" * 60)

# Set MLflow tracking
mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("Attrition_Prediction_IBM")

# Define models to test
models = {
    'Logistic_Regression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    'Random_Forest': RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(scale_pos_weight=3, eval_metric='logloss', random_state=42)
}

# Store results
results = []

# Train each model
for model_name, model in models.items():
    print(f"\n📊 Training {model_name}...")
    
    with mlflow.start_run(run_name=model_name):
        
        # Train
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)
        
        # Store results
        results.append({
            'Model': model_name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'ROC-AUC': roc_auc
        })
        
        # Log parameters
        mlflow.log_params(model.get_params())
        
        # Log metrics
        mlflow.log_metrics({
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc
        })
        
        # Log model
        mlflow.sklearn.log_model(model, model_name)
        
        # Feature importance for tree-based models
        if hasattr(model, 'feature_importances_'):
            # Log top 10 feature importances
            importances = model.feature_importances_
            top_indices = np.argsort(importances)[-10:]
            for idx in top_indices:
                if idx < len(feature_names):
                    mlflow.log_metric(f"imp_{feature_names[idx]}", importances[idx])
        
        print(f"   ✅ {model_name}: Acc={accuracy:.3f}, F1={f1:.3f}, AUC={roc_auc:.3f}")

# Create results dataframe
results_df = pd.DataFrame(results)
print("\n" + "=" * 60)
print("📊 MODEL COMPARISON RESULTS")
print("=" * 60)
print(results_df.to_string(index=False))

In [ ]:
# Cell 4 - Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Accuracy comparison
ax1 = axes[0, 0]
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC']
x = np.arange(len(results_df['Model']))
width = 0.15

for i, metric in enumerate(metrics):
    values = results_df[metric].values
    ax1.bar(x + i*width, values, width, label=metric)

ax1.set_xlabel('Model')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison')
ax1.set_xticks(x + width*2)
ax1.set_xticklabels(results_df['Model'])
ax1.legend(loc='lower right')
ax1.set_ylim(0, 1)

# Plot 2: Confusion Matrix for best model
best_model_idx = results_df['F1 Score'].argmax()
best_model_name = results_df.iloc[best_model_idx]['Model']
best_model = models[best_model_name]
y_pred_best = best_model.predict(X_test)

ax2 = axes[0, 1]
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2,
            xticklabels=['Stayed', 'Left'],
            yticklabels=['Stayed', 'Left'])
ax2.set_title(f'Confusion Matrix - {best_model_name}')
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Actual')

# Plot 3: ROC Curves
ax3 = axes[1, 0]
from sklearn.metrics import roc_curve

for model_name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    ax3.plot(fpr, tpr, label=f'{model_name} (AUC={auc:.3f})')

ax3.plot([0, 1], [0, 1], 'k--', label='Random')
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate')
ax3.set_title('ROC Curves Comparison')
ax3.legend()

# Plot 4: Feature Importance (if best model has it)
ax4 = axes[1, 1]
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    indices = np.argsort(importances)[-10:]  # Top 10
    
    # Get feature names safely
    top_features = []
    top_importances = []
    for idx in indices:
        if idx < len(feature_names):
            top_features.append(feature_names[idx])
            top_importances.append(importances[idx])
    
    y_pos = np.arange(len(top_features))
    ax4.barh(y_pos, top_importances)
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels(top_features)
    ax4.set_xlabel('Importance')
    ax4.set_title(f'Top 10 Features - {best_model_name}')
else:
    ax4.text(0.5, 0.5, 'Feature importance not available\nfor this model type',
             ha='center', va='center', transform=ax4.transAxes)

plt.tight_layout()
plt.savefig('../reports/figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 5 - Detailed analysis of best model
print("=" * 60)
print(f"🔍 DETAILED ANALYSIS - {best_model_name}")
print("=" * 60)

# Predictions
y_pred_best = best_model.predict(X_test)
y_proba_best = best_model.predict_proba(X_test)[:, 1]

# Classification report
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_best, 
                          target_names=['Stayed', 'Left']))

# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
npv = tn / (tn + fn)  # Negative predictive value

print(f"\n📊 Additional Metrics:")
print(f"   Specificity (True Negative Rate): {specificity:.3f}")
print(f"   NPV (Negative Predictive Value): {npv:.3f}")
print(f"   False Positive Rate: {fp/(fp+tn):.3f}")
print(f"   False Negative Rate: {fn/(fn+tp):.3f}")

# Risk stratification
y_proba_df = pd.DataFrame({
    'Actual': y_test.values,
    'Probability': y_proba_best
})
y_proba_df['Risk_Level'] = pd.cut(y_proba_df['Probability'], 
                                 bins=[0, 0.3, 0.6, 1.0],
                                 labels=['Low', 'Medium', 'High'])

print(f"\n🎯 Risk Stratification:")
print(y_proba_df.groupby('Risk_Level')['Actual'].agg(['count', 'mean']).round(3))

In [ ]:
# Cell 6 - Business impact simulation
print("=" * 60)
print("💰 BUSINESS IMPACT ANALYSIS")
print("=" * 60)

# Assumptions
avg_salary = df['MonthlyIncome'].mean() * 12
cost_per_hire = avg_salary * 0.5  # 50% of annual salary
current_attrition_rate = (df['Attrition'] == 'Yes').mean()
total_employees = len(df)

print(f"\n📈 ASSUMPTIONS:")
print(f"   • Average Annual Salary: ${avg_salary:,.0f}")
print(f"   • Cost per Hire (50% of salary): ${cost_per_hire:,.0f}")
print(f"   • Current Attrition Rate: {current_attrition_rate*100:.1f}%")
print(f"   • Total Employees: {total_employees}")

# Current cost
current_attrition_count = (df['Attrition'] == 'Yes').sum()
current_cost = current_attrition_count * cost_per_hire
print(f"\n💸 CURRENT ANNUAL ATTRITION COST:")
print(f"   • Employees leaving: {current_attrition_count}")
print(f"   • Total Cost: ${current_cost:,.0f}")

# Model impact scenarios
print(f"\n🎯 MODEL IMPACT SCENARIOS:")

# Scenario 1: 10% reduction in attrition
reduction_10 = current_attrition_count * 0.1
savings_10 = reduction_10 * cost_per_hire
print(f"\n   📉 10% Attrition Reduction:")
print(f"      • Employees saved: {reduction_10:.0f}")
print(f"      • Annual Savings: ${savings_10:,.0f}")

# Scenario 2: 20% reduction
reduction_20 = current_attrition_count * 0.2
savings_20 = reduction_20 * cost_per_hire
print(f"\n   📉 20% Attrition Reduction:")
print(f"      • Employees saved: {reduction_20:.0f}")
print(f"      • Annual Savings: ${savings_20:,.0f}")

# Scenario 3: Target high-risk only
high_risk_count = len(y_proba_df[y_proba_df['Risk_Level'] == 'High'])
print(f"\n   🎯 Target High-Risk Employees Only:")
print(f"      • High-risk employees identified: {high_risk_count}")
print(f"      • Potential annual savings: ${high_risk_count * cost_per_hire:,.0f}")

# Model performance value
print(f"\n💡 MODEL BUSINESS VALUE:")
print(f"   With {best_model_name}:")
print(f"   • Recall: {recall:.3f} (identifies {recall*100:.1f}% of leavers)")
print(f"   • Precision: {precision:.3f} ({precision*100:.1f}% accurate predictions)")

In [ ]:
# Cell 7 - Save the best model
print("=" * 60)
print("💾 SAVING BEST MODEL")
print("=" * 60)

# Save best model
best_model_path = '../models/best_model.pkl'
joblib.dump(best_model, best_model_path)
print(f"✅ Model saved to: {best_model_path}")

# Save feature names
feature_names_path = '../models/feature_names.pkl'
joblib.dump(feature_names, feature_names_path)
print(f"✅ Feature names saved to: {feature_names_path}")

# Register with MLflow
best_run = mlflow.search_runs(order_by=['metrics.f1_score DESC']).iloc[0]
best_run_id = best_run.run_id
print(f"\n📊 Best Run ID: {best_run_id}")
print(f"✅ Model registered in MLflow")

# Create model info file
model_info = {
    'best_model': best_model_name,
    'accuracy': accuracy,
    'f1_score': f1,
    'roc_auc': roc_auc,
    'features_count': len(feature_names),
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

model_info_path = '../models/model_info.json'
import json
with open(model_info_path, 'w') as f:
    json.dump(model_info, f, indent=4)
print(f"✅ Model info saved to: {model_info_path}")